# Shallow Neural Network Using Gradient Descent
---

In [1]:
import tensorflow as tf
import numpy as np

## Hyperparameters

In [2]:
EPOCHS = 1000

## Network Structure

Shallow Network

In [13]:
# the number of nodes in Input Layer: 2
# the number of nodes in Hidden Layer: 128 (activation: Sigmoid)
# the number of nodes in Output Layer: 10 (activation: Softmax)
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128, input_dim = 2, activation = 'sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation = 'softmax')
        
    def call(self, x, training = None, mask = None):
        x = self.d1(x)
        return self.d2(x)

## Training Loop

In [3]:
@tf.function # using the auto graph of TensorFlow
             # Implement all tensor operations in terms of Python Grammar.
             # By @tf.function, 'train_step()' is optimized.
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape: # recording all gradients calculated
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
    # partially differentiating 'loss' w.r.t each trainable variable
    gradients = tape.gradient(loss, model.trainable_variables)
    # inserting calculated gradients for update
    # update approach: selected optimizer for 'train_step()'
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # how to express overall losses (objective function) -> train_loss()
    train_loss(loss)
    # calculating an evaludation metric using ground truth (labels) and
    # predicted values
    train_metric(labels, predictions)

## Dataset and Preprocessing

In [6]:
np.random.seed(0)

pts = list() # features
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10, 2)) # within x [-8.0, 8.0]
                                                   # and y [-8.0, 8.0]
                                                   # grid..
                                                   # 10: ten points
                                                   # 2: (x, y) axes
# enumerate(): returns indices and corresponding values.
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)

pts = np.stack(pts, axis = 0).astype(np.float32) # to prevent errors
                                                 # (if float32 not set,
                                                 # automatically set as
                                                 # double - float64..)
                                                 # Unless set to be
                                                 # double initially
                                                 # 'and'
                                                 # if GPU applied,
                                                 # all operations
                                                 # basically implemented
                                                 # in terms of float32.
# no need 'astype(np.float32)' for labels (labels: integer)
labels = np.stack(labels, axis = 0)

# 'from_tensor_slices' method in 'Dataset' API: returns an iterable object.
# In the following, numpy object is inserted.
# A tensor object is also eligible for an input as well as a numpy object.
train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

## Generating a model

In [14]:
model = MyModel()

## Setting a Loss Function and an Optimization Algorithm

CrossEntropy, Adam Optimizer

In [11]:
# 'SparseCategoricalCrossentropy()': if only an index with '1'
# is designated..
# for a dataset one-hot encoding is applied to -> 'CategoricalCrossentropy()'
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## Setting a metric

Accuracy

In [12]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
# one of evaluation metrics
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

## Training

In [15]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
    print(f'Epoch {epoch + 1}, ' +
         f'Loss: {train_loss.result()}, ' +
         f'Accuracy: {train_accuracy.result() * 100}, ')

Epoch 1, Loss: 2.3554859161376953, Accuracy: 11.90000057220459, 
Epoch 2, Loss: 2.1480321884155273, Accuracy: 24.850000381469727, 
Epoch 3, Loss: 1.9891098737716675, Accuracy: 34.79999923706055, 
Epoch 4, Loss: 1.8636760711669922, Accuracy: 41.25, 
Epoch 5, Loss: 1.7592827081680298, Accuracy: 47.47999954223633, 
Epoch 6, Loss: 1.6734800338745117, Accuracy: 51.93333435058594, 
Epoch 7, Loss: 1.5997918844223022, Accuracy: 55.80000305175781, 
Epoch 8, Loss: 1.535377025604248, Accuracy: 58.412498474121094, 
Epoch 9, Loss: 1.4777721166610718, Accuracy: 60.86666488647461, 
Epoch 10, Loss: 1.4266853332519531, Accuracy: 62.76000213623047, 
Epoch 11, Loss: 1.379510760307312, Accuracy: 64.69091033935547, 
Epoch 12, Loss: 1.3364185094833374, Accuracy: 66.2249984741211, 
Epoch 13, Loss: 1.2973161935806274, Accuracy: 67.73846435546875, 
Epoch 14, Loss: 1.2608909606933594, Accuracy: 68.83570861816406, 
Epoch 15, Loss: 1.2272834777832031, Accuracy: 69.913330078125, 
Epoch 16, Loss: 1.196079134941101,

Epoch 127, Loss: 0.46776100993156433, Accuracy: 86.20551300048828, 
Epoch 128, Loss: 0.4662933051586151, Accuracy: 86.22969055175781, 
Epoch 129, Loss: 0.46486684679985046, Accuracy: 86.25503540039062, 
Epoch 130, Loss: 0.46342945098876953, Accuracy: 86.28076934814453, 
Epoch 131, Loss: 0.4620361924171448, Accuracy: 86.30152893066406, 
Epoch 132, Loss: 0.4606859087944031, Accuracy: 86.3272705078125, 
Epoch 133, Loss: 0.45934879779815674, Accuracy: 86.3481216430664, 
Epoch 134, Loss: 0.4580545425415039, Accuracy: 86.36566925048828, 
Epoch 135, Loss: 0.4568830728530884, Accuracy: 86.38666534423828, 
Epoch 136, Loss: 0.45552125573158264, Accuracy: 86.40808868408203, 
Epoch 137, Loss: 0.45418378710746765, Accuracy: 86.429931640625, 
Epoch 138, Loss: 0.4528941810131073, Accuracy: 86.4521713256836, 
Epoch 139, Loss: 0.4516351819038391, Accuracy: 86.47481536865234, 
Epoch 140, Loss: 0.45037534832954407, Accuracy: 86.49214172363281, 
Epoch 141, Loss: 0.4491467773914337, Accuracy: 86.5078048706

Epoch 372, Loss: 0.3346438705921173, Accuracy: 88.18978881835938, 
Epoch 373, Loss: 0.3344321846961975, Accuracy: 88.19248962402344, 
Epoch 374, Loss: 0.3342512547969818, Accuracy: 88.1954574584961, 
Epoch 375, Loss: 0.33403635025024414, Accuracy: 88.19973754882812, 
Epoch 376, Loss: 0.33382532000541687, Accuracy: 88.20106506347656, 
Epoch 377, Loss: 0.3336256146430969, Accuracy: 88.20478057861328, 
Epoch 378, Loss: 0.3334163427352905, Accuracy: 88.20873260498047, 
Epoch 379, Loss: 0.3332180678844452, Accuracy: 88.2108154296875, 
Epoch 380, Loss: 0.33303502202033997, Accuracy: 88.21394348144531, 
Epoch 381, Loss: 0.3328462243080139, Accuracy: 88.2170639038086, 
Epoch 382, Loss: 0.3326456844806671, Accuracy: 88.2196273803711, 
Epoch 383, Loss: 0.33244019746780396, Accuracy: 88.22428131103516, 
Epoch 384, Loss: 0.3322567641735077, Accuracy: 88.22760772705078, 
Epoch 385, Loss: 0.3320584297180176, Accuracy: 88.2301254272461, 
Epoch 386, Loss: 0.3318808674812317, Accuracy: 88.2347183227539

Epoch 496, Loss: 0.3152220845222473, Accuracy: 88.49798583984375, 
Epoch 497, Loss: 0.3150905966758728, Accuracy: 88.49939727783203, 
Epoch 498, Loss: 0.3149704337120056, Accuracy: 88.50100708007812, 
Epoch 499, Loss: 0.3148557245731354, Accuracy: 88.50420379638672, 
Epoch 500, Loss: 0.3147331178188324, Accuracy: 88.5060043334961, 
Epoch 501, Loss: 0.31461572647094727, Accuracy: 88.50698852539062, 
Epoch 502, Loss: 0.3144914209842682, Accuracy: 88.5087661743164, 
Epoch 503, Loss: 0.31439095735549927, Accuracy: 88.51093292236328, 
Epoch 504, Loss: 0.31428852677345276, Accuracy: 88.51170349121094, 
Epoch 505, Loss: 0.3141654431819916, Accuracy: 88.51386260986328, 
Epoch 506, Loss: 0.31407687067985535, Accuracy: 88.51561737060547, 
Epoch 507, Loss: 0.31396952271461487, Accuracy: 88.5173568725586, 
Epoch 508, Loss: 0.313847154378891, Accuracy: 88.5188980102539, 
Epoch 509, Loss: 0.31372347474098206, Accuracy: 88.5212173461914, 
Epoch 510, Loss: 0.313607394695282, Accuracy: 88.52392578125, 

Epoch 741, Loss: 0.2945021688938141, Accuracy: 88.87125396728516, 
Epoch 742, Loss: 0.2944353520870209, Accuracy: 88.87237548828125, 
Epoch 743, Loss: 0.2943689227104187, Accuracy: 88.87361907958984, 
Epoch 744, Loss: 0.294314980506897, Accuracy: 88.875, 
Epoch 745, Loss: 0.2942569851875305, Accuracy: 88.8762435913086, 
Epoch 746, Loss: 0.29419296979904175, Accuracy: 88.87788391113281, 
Epoch 747, Loss: 0.2941288352012634, Accuracy: 88.87937927246094, 
Epoch 748, Loss: 0.294064998626709, Accuracy: 88.88047790527344, 
Epoch 749, Loss: 0.2940119206905365, Accuracy: 88.88224029541016, 
Epoch 750, Loss: 0.2939477264881134, Accuracy: 88.88333129882812, 
Epoch 751, Loss: 0.29388749599456787, Accuracy: 88.88508605957031, 
Epoch 752, Loss: 0.2938261032104492, Accuracy: 88.88670349121094, 
Epoch 753, Loss: 0.2937585413455963, Accuracy: 88.88751983642578, 
Epoch 754, Loss: 0.2937059998512268, Accuracy: 88.88886260986328, 
Epoch 755, Loss: 0.29365068674087524, Accuracy: 88.88927459716797, 
Epoch 

Epoch 988, Loss: 0.2828969955444336, Accuracy: 89.16893005371094, 
Epoch 989, Loss: 0.2828645706176758, Accuracy: 89.16946411132812, 
Epoch 990, Loss: 0.2828240990638733, Accuracy: 89.17050170898438, 
Epoch 991, Loss: 0.2827922999858856, Accuracy: 89.17154693603516, 
Epoch 992, Loss: 0.28275471925735474, Accuracy: 89.17237854003906, 
Epoch 993, Loss: 0.28271403908729553, Accuracy: 89.17321014404297, 
Epoch 994, Loss: 0.28268325328826904, Accuracy: 89.17384338378906, 
Epoch 995, Loss: 0.2826510965824127, Accuracy: 89.17537689208984, 
Epoch 996, Loss: 0.28261178731918335, Accuracy: 89.17671203613281, 
Epoch 997, Loss: 0.28257736563682556, Accuracy: 89.17742919921875, 
Epoch 998, Loss: 0.28254398703575134, Accuracy: 89.17845153808594, 
Epoch 999, Loss: 0.2825019359588623, Accuracy: 89.17938232421875, 
Epoch 1000, Loss: 0.2824624180793762, Accuracy: 89.18030548095703, 


## Saving the dataset and the trained parameters

In [16]:
# saving the dataset
# 'np.savez': enables to save multiple numpy objects.
# 'compressed': compressing! (for reducing the size of a file)
np.savez_compressed('ShallowNN_GD_dataset.npz', inputs = pts, labels = labels)

# saving the trained parameters
W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
# to match the convention TensorFlow uses, tranpose it!
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ShallowNN_GD_parameters.npz',
                   W_h = W_h,
                   b_h = b_h,
                   W_o = W_o,
                   b_o = b_o)